## 한국어 키워드 추출

In [1]:
#!pip install sentence_transformers
# !pip install konlpy

모듈 불러오기

In [20]:
import numpy as np
import itertools
import pandas as pd

from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer


예시

In [3]:
doc = """
청탁 대가 명목으로 사업가로부터 거액을 수수한 혐의를 받는 이정근 전 더불어민주당 사무부총장이 
30일 오전 서울 서초동 중앙지법에서 열린 구속 전 피의자심문(영장실질심사)에 출석하고 있다. 
연합뉴스이정근 전 더불어민주당 사무부총장이 30일 늦은 저녁 구속됐다.
▶김상우 서울중앙지법 영장 전담 부장판사는 여러 청탁을 들어주는 대가로 사업가로부터 10억원대 금품을 수수한 혐의를 받는 이정근 전 사무부총장에 대해 
이날 오전 구속 전 피의자 심문(영장실질심사)을 진행한 후 증거인멸 및 도망 우려를 이유로 구속영장 발부 결정을 내렸다.
이정근 전 사무부총장은 지난 2019년 12월부터 올해 1월까지 100억원대 정부 에너지 기금 배정, 마스크 사업 관련 인허가 및 공공기관 납품, 한국남부발전 임직원 승진 등에 대한 알선 등 
명목으로 사업가 박모씨로부터 수십 차례에 걸쳐 총 9억5천만원을 수수한 혐의(특가법상 알선수재·변호사법 위반)를 받는다.
또 21대 총선이 있었던 2020년 2∼4월에도 박씨로부터 선거 비용 명목으로 수 회에 걸쳐 3억3천만원의 불법 정치자금을 받은 혐의(정치자금법 위반)도 받고 있다.
검찰은 알선 대가로 받은 돈과 불법 정치자금이 일부 겹친다고 보고 이정근 전 사무부총장의 총 수수 금액을 10억1천만원으로 판단했다.
이에 대해 이정근 전 사무부총장은 박씨와의 자금 거래가 채권 및 채무 관계일 뿐이라면서 불법성이 없다고 반박하고 있다. 
그는 이날 영장심사에 출석하면서 언론에 "억울함을 잘 밝히겠다"고 했다.▶이정근 전 사무부총장은 1962년 전북 군산 태생으로 올해 나이 61세이다.
MBC PD수첩 취재리서처, 더불어민주당 싱크탱크인 민주연구원 이사, 더불어민주당 중앙당 부대변인 등을 지냈다. 
19대 대선 당시 문재인 후보 선대위 본부장, 20대 대선 당시 이재명 후보 선대위 부본부장을 맡기도 했다.
2016년 20대 총선 ,2020년 21대 총선, 2022년 3월 재보궐선거 등 3차례 서울 서초갑 국회의원 선거에 나서 모두 2위로 낙선했다.
그 사이인 2018년 7회 지방선거 서초구청장 선거에도 출마했으나 역시 2위로 떨어졌다.
"""

형태소 분석기를 통한 명사 추출

In [4]:
okt = Okt()

##### 자바 오류 해결  
https://stricky.tistory.com/398

In [5]:
tokenized_doc = okt.pos(doc)
tokenized_nouns = ' '.join([word[0] for word in tokenized_doc if word[1] == 'Noun'])

# for word in tokenized_doc:
#     if word[1] == 'Noun':
#         word[0]

In [6]:
tokenized_nouns

'청탁 대가 명목 사업가 로부터 거액 혐의 이정근 전 민주당 사무 부총 오전 서울 서초동 중앙 지법 구속 전 피의자 심문 영장 실질 심사 출석 연합뉴스 이정근 전 민주당 사무 부총 저녁 구속 김상우 서울중앙지법 영장 전담 부장 판사 여러 청탁 대가 사업가 로부터 금품 혐의 이정근 전 사무 부총 대해 날 오전 구속 전 피의자 심문 영장 실질 심사 진행 후 증거 멸 및 도망 우려 이유 구속영장 발부 결정 이정근 전 사무 부총 지난 올해 정부 에너지 기금 배정 마스크 사업 관련 인 허가 및 공공기관 납품 한국 남부 발전 임 직원 승진 등 대한 알선 등 명목 사업가 박모 로부터 수십 차례 총 혐의 특가법 알선 수재 변호 사법 위반 를 또 총선 박씨 로부터 선거 비용 명목 수 회 불법 정치자금 혐의 정치자금 법 위반 도 검찰 알선 대가 돈 불법 정치자금 일부 겹 보고 이정근 전 사무 부총 총 수수 금액 판단 이 대해 이정근 전 사무 부총 박씨 자금 거래 채권 및 채무 관계 일 불법 반박 그 날 영장 심사 출석 언론 고 이정근 전 사무 부총 전북 군산 태생 올해 나이 세이 수첩 취재 리 처 민주당 싱크탱크 민주 연구원 이사 민주당 중앙 부대 변인 등 대선 당시 문재인 후보 선 대위 본부장 대선 당시 이재명 후보 선 대위 부본 부장 총선 총선 재보궐선거 등 차례 서울 서초 갑 국회의원 선거 모두 위로 낙선 그 사이 지방선거 서초구청장 선거 출마 역시 위로'

In [7]:
tokenized_doc

[('\n', 'Foreign'),
 ('청탁', 'Noun'),
 ('대가', 'Noun'),
 ('명목', 'Noun'),
 ('으로', 'Josa'),
 ('사업가', 'Noun'),
 ('로부터', 'Noun'),
 ('거액', 'Noun'),
 ('을', 'Josa'),
 ('수수한', 'Adjective'),
 ('혐의', 'Noun'),
 ('를', 'Josa'),
 ('받는', 'Verb'),
 ('이정근', 'Noun'),
 ('전', 'Noun'),
 ('더불어', 'Verb'),
 ('민주당', 'Noun'),
 ('사무', 'Noun'),
 ('부총', 'Noun'),
 ('장이', 'Suffix'),
 ('30일', 'Number'),
 ('오전', 'Noun'),
 ('서울', 'Noun'),
 ('서초동', 'Noun'),
 ('중앙', 'Noun'),
 ('지법', 'Noun'),
 ('에서', 'Josa'),
 ('열린', 'Verb'),
 ('구속', 'Noun'),
 ('전', 'Noun'),
 ('피의자', 'Noun'),
 ('심문', 'Noun'),
 ('(', 'Punctuation'),
 ('영장', 'Noun'),
 ('실질', 'Noun'),
 ('심사', 'Noun'),
 (')', 'Punctuation'),
 ('에', 'Josa'),
 ('출석', 'Noun'),
 ('하고', 'Josa'),
 ('있다', 'Adjective'),
 ('.', 'Punctuation'),
 ('연합뉴스', 'Noun'),
 ('이정근', 'Noun'),
 ('전', 'Noun'),
 ('더불어', 'Verb'),
 ('민주당', 'Noun'),
 ('사무', 'Noun'),
 ('부총', 'Noun'),
 ('장이', 'Suffix'),
 ('30일', 'Number'),
 ('늦은', 'Verb'),
 ('저녁', 'Noun'),
 ('구속', 'Noun'),
 ('됐다', 'Verb'),
 ('.', 'Punctuati

In [8]:
# n-gram : 단어의 묶음
# ngram_range: tuple (min_n, max_n), default=(1, 1)
# n-gram은 단어의 묶음을 2개부터 3개까지 설정(인덱스 부여)
# https://bit.ly/3V99fij (참고)
n_gram_range = (2, 3)


# CountVectorizer
# 문서를 토큰 리스트로 변환
# 각 문서에서 토큰의 출현 빈도
# 각 문서를 BOW 인코딩 벡터로 변환
count = CountVectorizer(ngram_range=n_gram_range).fit([tokenized_nouns])


candidates = count.get_feature_names_out()

In [9]:
# SBERT를 통해서 수치화, 한국어를 포함하고 있는 다국어 SBERT 로드
# https://bit.ly/3EpuKFs (참고)
model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')
doc_embedding = model.encode([doc])
candidate_embeddings = model.encode(candidates)

In [14]:
top_n = 5
distances = cosine_similarity(doc_embedding, candidate_embeddings)
keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]
print(keywords)

['김상우 서울중앙지법', '대선 당시 문재인', '당시 문재인 후보', '수십 차례 혐의', '구속 김상우 서울중앙지법']


### 중간결론 : 비슷한 의미의 키워드들이 리턴됨.
 - 다양한 키워드를 아래 알고리즘을 통해 얻어보기
 - Max Sum Similarity
 - Maximal Marginal Relevance

## **Max Sum Similarity**
- 데이터 쌍 사이의 최대 합 거리는 데이터 쌍 간의 거리가 최대화되는 데이터 쌍으로 정의
- 후보 간의 유사성을 최소화하면서 문서와의 후보 유사성을 극대화

In [31]:
def max_sum_sim(doc_embedding, candidate_embeddings, words, top_n, nr_candidates):
    # 문서와 각 키워드들 간의 유사도
    distances = cosine_similarity(doc_embedding, candidate_embeddings)

    # 각 키워드들 간의 유사도
    distances_candidates = cosine_similarity(candidate_embeddings, 
                                            candidate_embeddings)

    # 코사인 유사도에 기반하여 키워드들 중 상위 top_n개의 단어를 pick.
    words_idx = list(distances.argsort()[0][-nr_candidates:])
    words_vals = [candidates[index] for index in words_idx]
    
    # (참고) numpy ix_  https://bit.ly/3CBFrUa ,  explain: can quickly construct index arrays that will index the cross product.
    distances_candidates = distances_candidates[np.ix_(words_idx, words_idx)]

    # 각 키워드들 중에서 가장 덜 유사한 키워드들간의 조합을 계산
    min_sim = np.inf
    candidate = None
    # combination(iterable, r) => iterable에서 원소 개수가 r개인 조합 뽑기
    for combination in itertools.combinations(range(len(words_idx)), top_n):
        sim = sum([distances_candidates[i][j] for i in combination for j in combination if i != j])
        if sim < min_sim:
            candidate = combination
            min_sim = sim

    return [words_vals[idx] for idx in candidate]

In [34]:
# 상위 10개의 키워드를 선택하고 이 10개 중에서 서로 가장 유사성이 낮은 5개를 선택
# 낮은 nr_candidates를 설정해 출력된 5개 키워드 결과는 기존의 코사인 유사도만 사용한 것과 유사함
max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n=3, nr_candidates=10)

['대가 불법 정치자금', '당시 문재인 후보', '수십 차례 혐의']

In [35]:
# 상대적으로 높은 nr_candidates는 다양한 키워드 5개를 도출
max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n=3, nr_candidates=30)

['재보궐선거 차례 서울', '대가 불법 정치자금', '수십 차례 혐의']

#### 결론 : 상위 10개 키워드중 유사성이 낮은 n개를 선택해 다양한 키워드 도출